In [1]:
from dotenv import load_dotenv
load_dotenv()
from functions.disturbances import (
    replace_characters,
    add_characters,
    add_random_words,
    remplace_named_entities,
    change_order,
    use_negation,
    introducce_demografic_context
)
from functions.utils import request_to_bard, request_to_bing
from functions.metamorphic import (
    input_equivalence_mrt,
    equivalence_set_mrt,
    get_distance_answers,
    distance_set_mrt
)
    
    

c:\Users\tcame\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
models = [
    ('summarize', 'facebook/bart-large-cnn'),
    ('summarize', 'google/pegasus-large'),
    ('summarize', 'marianna13/flan-t5-base-summarization'),
    ('toxic', 's-nlp/roberta_toxicity_classifier'),
    ('toxic', 'citizenlab/distilbert-base-multilingual-cased-toxicity', 'inputs'),
    ('toxic', 'martin-ha/toxic-comment-model'),
    ('spam', 'rafacost/bert_base_pt_en_cased_email_spam'),
    ('spam', 'h-e-l-l-o/email-spam-classification-merged'),
    ('spam', 'dima806/email-spam-detection-roberta'),
    ('translate', 't5-base'),
    ('translate', 'allenai/wmt16-en-de-12-1'),
    ('translate', 'facebook/wmt19-en-de'),
    ('fillmask', 'bert-base-uncased', '[MASK]'),
    ('fillmask', 'vinai/bertweet-base', '<mask>'),
    ('fillmask', 'roberta-base', '<mask>'),
]

COMPROBAR QUE BARD FUNCIONA

In [3]:
print(request_to_bard("What is the capital of Spain?"))

Madrid


COMPROAR QUE BING FUNCIONA

In [4]:
print(await request_to_bing("What is the capital of Spain?"))

The capital of Spain is Madrid..


PERTURBACIONES

In [5]:
print(replace_characters("Hello how are you? It's been a while since you and I met.", 5))

TelSo hdw are Rou? Qt's been a while since yqR LnE I met.


In [6]:
print(add_characters("Hello how are you? It's been a while since you and I met.", 4))

Heljlo hXowl iare yYou? chIt's been a whiBle since yAou andI EI met.


In [29]:
text_random_words = await add_random_words("Hello how are you? It's been a while since you and I met.")
print(text_random_words)

Hello, this is Bing. I'm an AI and don't have feelings, but I'm here to help you. Yes, it seems like it's been a while. How can I assist you today?


In [9]:
text_remplace_named_entities = await remplace_named_entities("Tomas and Maria are from Spain. They are 20 years old. They are married. They have two children.")
print(text_remplace_named_entities)

Tomas and Maria are a married couple from Spain. Both of them are 20 years old and they have two children.


In [27]:
text_change_order = await change_order("is this a question?")
print(text_change_order)

Yes, it is a question.


In [30]:
text_use_negation = await use_negation("Hello how are you? It's been a while since you and I met. I like to eat pizza.")
print(text_use_negation)

Hello, I'm here to assist you. It's true that we haven't interacted in a while. It's great to hear that you enjoy eating pizza. How can I assist you further?


In [12]:
text_demografic_context = await introducce_demografic_context("What is the capital of Spain?")
print(text_demografic_context)

The capital of Spain is Madrid..


PLANTILLAS RELACIONES METAMÓRFICAS

In [25]:
original_imput = "You are an idiot"
disturbed_input = add_characters("You are an idiot",2)
res = input_equivalence_mrt(models[3], original_imput, disturbed_input)
print(res)

True


In [16]:
print(await equivalence_set_mrt(models[8], "Buy a car in my store and get a 20 discount"))

False


In [19]:
texto1 = "Hola que tal"
texto2 = "Hola que tal estas"
print(await get_distance_answers(texto1,texto2,0))

83.5


In [42]:
res = await distance_set_mrt("What is the capital of Spain?", models[9])
print(res)

Exception: Response status code is not 200. Response Status is 404